# Part 2: Training

In this part we will show how using MLRun's **Feature Store** we can easily define a **Feature Vector** and create the dataset we need to run our training process.  
By the end of this tutorial you’ll learn how to:
- Combine multiple data sources to a single Feature Vector
- Create training dataset
- Create a model using an MLRun Hub function

In [1]:
project_name = 'fraud-demo'

In [2]:
import mlrun

# Initialize the MLRun project object
project = mlrun.get_or_create_project(project_name, context="./", user_project=True)

> 2022-06-19 13:02:28,474 [info] loaded project fraud-demo from MLRun DB


## Step 1 - Create a Feature Vector  
In this section we will create our Feature Vector.  
The Feature vector will have a `name` so we can reference to it later via the URI or our serving function, and a list of `features` from the available FeatureSets.  We can add a feature from a feature set by adding `<FeatureSet>.<Feature>` to the list, or add `<FeatureSet>.*` to add all the FeatureSet's available features.  

By default, the first FeatureSet in the feature list will act as the spine. meaning that all the other features will be joined to it.  
For example, in this instance we use the early_sense sensor data as our spine, so for each early_sense event we will create produce a row in the resulted Feature Vector.

In [3]:
# Define the list of features we will be using
features = ['transactions.amount_max_2h', 
            'transactions.amount_sum_2h', 
            'transactions.amount_count_2h',
            'transactions.amount_avg_2h', 
            'transactions.amount_max_12h', 
            'transactions.amount_sum_12h',
            'transactions.amount_count_12h', 
            'transactions.amount_avg_12h', 
            'transactions.amount_max_24h',
            'transactions.amount_sum_24h', 
            'transactions.amount_count_24h', 
            'transactions.amount_avg_24h',
            'transactions.es_transportation_count_14d', 
            'transactions.es_health_count_14d',
            'transactions.es_otherservices_count_14d', 
            'transactions.es_food_count_14d',
            'transactions.es_hotelservices_count_14d', 
            'transactions.es_barsandrestaurants_count_14d',
            'transactions.es_tech_count_14d', 
            'transactions.es_sportsandtoys_count_14d',
            'transactions.es_wellnessandbeauty_count_14d', 
            'transactions.es_hyper_count_14d',
            'transactions.es_fashion_count_14d', 
            'transactions.es_home_count_14d', 
            'transactions.es_travel_count_14d', 
            'transactions.es_leisure_count_14d',
            'transactions.gender_F',
            'transactions.gender_M',
            'transactions.step', 
            'transactions.amount', 
            'transactions.timestamp_hour',
            'transactions.timestamp_day_of_week',
            'events.*']

In [4]:
# Import MLRun's Feature Store
import mlrun.feature_store as fstore

# Define the feature vector name for future reference
fv_name = 'transactions-fraud'

# Define the feature vector using our Feature Store (fstore)
transactions_fv = fstore.FeatureVector(fv_name, 
                          features, 
                          label_feature="labels.label",
                          description='Predicting a fraudulent transaction')

# Save the feature vector in the Feature Store
transactions_fv.save()

## Step 2 - Preview the Feature Vector Data

Obtain the values of the features in the feature vector, to ensure the data appears as expected

In [5]:
# Import the Parquet Target so we can directly save our dataset as a file
from mlrun.datastore.targets import ParquetTarget

# Get offline feature vector as dataframe and save the dataset to parquet
train_dataset = fstore.get_offline_features(fv_name, target=ParquetTarget())

> 2022-06-19 13:02:29,517 [info] wrote target: {'name': 'parquet', 'kind': 'parquet', 'path': 'v3io:///projects/fraud-demo-dani/FeatureStore/transactions-fraud/parquet/vectors/transactions-fraud-latest.parquet', 'status': 'ready', 'updated': '2022-06-19T13:02:29.517761+00:00', 'size': 1637700}


In [6]:
# Preview our dataset
train_dataset.to_dataframe().tail(5)

,amount_max_2h,amount_sum_2h,amount_count_2h,amount_avg_2h,amount_max_12h,amount_sum_12h,amount_count_12h,amount_avg_12h,amount_max_24h,amount_sum_24h,...,gender_F,gender_M,step,amount,timestamp_hour,timestamp_day_of_week,event_details_change,event_login,event_password_change,label
49996,22.40,61.48,4.0,15.370000,48.95,110.43,5.0,22.086000,48.95,136.43,...,0,1,63,22.40,12,6,0.0,0.0,1.0,0
49997,35.27,35.27,1.0,35.270000,124.35,240.44,6.0,40.073333,124.35,240.44,...,1,0,113,35.27,12,6,0.0,1.0,0.0,0
49998,77.76,162.28,4.0,40.570000,77.76,737.81,23.0,32.078696,77.76,1611.80,...,0,1,6,35.65,12,6,0.0,0.0,1.0,0
49999,26.74,47.14,2.0,23.570000,30.04,125.13,6.0,20.855000,64.06,303.32,...,1,0,132,26.74,12,6,0.0,0.0,1.0,0
50000,75.11,253.56,7.0,36.222857,166.18,996.76,27.0,36.917037,204.29,2083.14,...,1,0,103,20.70,13,6,1.0,0.0,0.0,0


## Step 3 - Train Models and Choose Highest Accuracy

With MLRun, one can easily train different models and compare the results. In the code below, we train 3 different models,
each uses a different algorithm (random forest, XGBoost, adabost), and choose the model with the highest accuracy

In [7]:
# Import the Sklearn classifier function from the functions hub
classifier_fn = mlrun.import_function('hub://sklearn-classifier')

In [8]:
# Prepare the parameters list for the training function
# We will be using 3 different models
training_params = {"model_name": ['transaction_fraud_rf', 
                                  'transaction_fraud_xgboost', 
                                  'transaction_fraud_adaboost'],
              
              "model_pkg_class": ['sklearn.ensemble.RandomForestClassifier',
                                  'sklearn.ensemble.GradientBoostingClassifier',
                                  'sklearn.ensemble.AdaBoostClassifier']}

# Define the training task, including our feature vector, label and hyperparams definitions
train_task = mlrun.new_task('training', 
                      inputs={'dataset': transactions_fv.uri},
                      params={'label_column': 'label'}
                     )

train_task.with_hyper_params(training_params, strategy='list', selector='max.accuracy')

# Specify our cluster image
classifier_fn.spec.image = 'mlrun/mlrun'

# Run training
classifier_fn.run(train_task, local=False)

> 2022-06-19 13:02:30,070 [info] starting run training uid=a40ef88b56cc41e2a03afab1ed237e0b DB=http://mlrun-api:8080
> 2022-06-19 13:02:30,260 [info] Job is running in the background, pod: training-p596c
> 2022-06-19 13:02:50,799 [info] best iteration=1, used criteria max.accuracy
> 2022-06-19 13:02:51,440 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
fraud-demo-dani,...ed237e0b,0,Jun 19 13:02:34,completed,training,v3io_user=danikind=jobowner=danimlrun/client_version=1.0.4,dataset,label_column=label,best_iteration=1accuracy=0.9964788732394366test-error=0.0035211267605633804rocauc=0.9485119282984051brier_score=0.003612535211267606f1-score=0.8148148148148148precision_score=0.9166666666666666recall_score=0.7333333333333333,test_setprobability-calibrationconfusion-matrixfeature-importancesprecision-recall-binaryroc-binarymodeliteration_resultsparallel_coordinates


> 2022-06-19 13:02:59,734 [info] run executed, status=completed


## Step 4 - Perform Feature Selection

As part of our data science process we will try and reduce the training dataset's size to get rid of bad or unuseful features and save computation time.

We will use our ready-made feature selection function from our hub [`hub://feature_selection`](https://github.com/mlrun/functions/blob/development/feature_selection/feature_selection.ipynb) to select the best features to keep on a sample from our dataset and run the function on that.


In [9]:
feature_selection_fn = mlrun.import_function('hub://feature_selection')

feature_selection_run = feature_selection_fn.run(
            params={'sample_ratio':0.25,
                    'output_vector_name':fv_name + "-short",
                   'ignore_type_errors': True},
    
            inputs={'df_artifact': transactions_fv.uri},
            name='feature_extraction',
            handler='feature_selection',
    local=False)

> 2022-06-19 13:03:00,149 [info] starting run feature_extraction uid=743fa05d57924429acee5a08da450251 DB=http://mlrun-api:8080
> 2022-06-19 13:03:00,321 [info] Job is running in the background, pod: feature-extraction-9gx9l
> 2022-06-19 13:03:08,564 [info] Couldn't calculate chi2 because of: Input X must be non-negative.
> 2022-06-19 13:03:13,003 [info] votes needed to be selected: 3
> 2022-06-19 13:03:13,582 [info] wrote target: {'name': 'parquet', 'kind': 'parquet', 'path': 'v3io:///projects/fraud-demo-dani/FeatureStore/transactions-fraud-short/parquet/vectors/transactions-fraud-short-latest.parquet', 'status': 'ready', 'updated': '2022-06-19T13:03:13.582737+00:00', 'size': 309440}
> 2022-06-19 13:03:13,692 [info] run executed, status=completed
Liblinear failed to converge, increase the number of iterations.
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
fraud-demo-dani,...da450251,0,Jun 19 13:03:04,completed,feature_extraction,v3io_user=danikind=jobowner=danimlrun/client_version=1.0.4host=feature-extraction-9gx9l,df_artifact,sample_ratio=0.25output_vector_name=transactions-fraud-shortignore_type_errors=True,top_features_vector=store://feature-vectors/fraud-demo-dani/transactions-fraud-short,f_classifmutual_info_classiff_regressionLinearSVCLogisticRegressionExtraTreesClassifierfeature_scoresmax_scaled_scores_feature_scoresselected_features_countselected_features


> 2022-06-19 13:03:15,753 [info] run executed, status=completed


In [10]:
mlrun.get_dataitem(feature_selection_run.outputs['top_features_vector']).as_df().tail(5)

,amount_max_2h,amount_sum_2h,amount_count_2h,amount_avg_2h,amount_max_12h,label
49996,22.40,61.48,4.0,15.370000,48.95,0
49997,35.27,35.27,1.0,35.270000,124.35,0
49998,77.76,162.28,4.0,40.570000,77.76,0
49999,26.74,47.14,2.0,23.570000,30.04,0
50000,75.11,253.56,7.0,36.222857,166.18,0


## Step 5 - Train our models with top features

Following the feature selection, we train new models using the resultant features. We can observe the accuracy and other results remain high
meaning we get a model that requires less features to be accurate and thus less error-prone.

In [11]:
# Defining our training task, including our feature vector, label and hyperparams definitions
ensemble_train_task = mlrun.new_task('training', 
                      inputs={'dataset': feature_selection_run.outputs['top_features_vector']},
                      params={'label_column': 'label'}
                     )
ensemble_train_task.with_hyper_params(training_params, strategy='list', selector='max.accuracy')

classifier_fn.run(ensemble_train_task)

> 2022-06-19 13:03:15,838 [info] starting run training uid=de073c1e7063427c8f4b4eb60fc89feb DB=http://mlrun-api:8080
> 2022-06-19 13:03:16,017 [info] Job is running in the background, pod: training-v6stc
> 2022-06-19 13:03:29,209 [info] best iteration=1, used criteria max.accuracy
> 2022-06-19 13:03:29,741 [info] run executed, status=completed
final state: completed


project,uid,iter,start,state,name,labels,inputs,parameters,results,artifacts
fraud-demo-dani,...0fc89feb,0,Jun 19 13:03:20,completed,training,v3io_user=danikind=jobowner=danimlrun/client_version=1.0.4,dataset,label_column=label,best_iteration=1accuracy=0.9939898289412853test-error=0.006010171058714748rocauc=0.8803831466305261brier_score=0.005478554657311804f1-score=0.45833333333333326precision_score=0.7333333333333333recall_score=0.3333333333333333,test_setprobability-calibrationconfusion-matrixfeature-importancesprecision-recall-binaryroc-binarymodeliteration_resultsparallel_coordinates


> 2022-06-19 13:03:35,379 [info] run executed, status=completed


## Done!

You've completed Part 2 of the model training with the feature store.
Proceed to [Part 3](03-deploy-serving-model.ipynb) to learn how to deploy and monitor the model.